# Stored Procedure 預存程序

<img src='img/1.png'>

# 為何要學習  Stored Procedure 這項技術

# 建立  Stored Procedure

In [ ]:
CREATE PROC ProdOrders
AS
SELECT Position, sum(AnnualSalary) as TotalSalary
FROM Employee
WHERE Address = 'taipei'
GROUP BY Position
GO

# 執行 Stored Procedure

### 方法一: EXEC 預存程序名

In [ ]:
EXEC ProdOrders

### 方法二: 直接輸入預存程序名

In [ ]:
ProdOrders

# 修改 Stored Procedure

In [ ]:
ALTER PROC ProdOrders
AS
SELECT Position, sum(AnnualSalary) as TotalSalary
FROM Employee
WHERE Address = 'taichung'
GROUP BY Position
GO

# 預存程序傳入參數

In [ ]:
CREATE PROC ProdOrdersID
(
@Address varchar(10)
)
AS
SELECT Position, sum(AnnualSalary) as TotalSalary
FROM Employee
WHERE Address = @Address
GROUP BY Position
GO

In [ ]:
EXEC ProdOrdersID "taipei"

# 建立 sp 的細節

### 範例 1 - 曲線

In [ ]:
ALTER PROCEDURE   [dbo].[tag_battery]
	-- Add the parameters for the stored procedure here
	@begin date,
	@end date,
	@id nvarchar(5),
	@begin_h nvarchar(2)  = '0',
	@end_h nvarchar(2)  = '23'
AS
BEGIN
	-- SET NOCOUNT ON added to prevent extra result sets from
	-- interfering with SELECT statements.
	SET NOCOUNT ON;

    -- Insert statements for procedure here
	declare @year    nvarchar(5) = DATEPART(YEAR, @begin)
	declare @begin_m nvarchar(3) = DATEPART(MONTH, @begin)
	declare @begin_d nvarchar(3) = DATEPART(DAY, @begin)
	declare @end_m   nvarchar(3) = DATEPART(MONTH, @end)
	declare @end_d   nvarchar(3) = DATEPART(DAY, @end)

	set @id = ltrim(rtrim(@id))

	declare @gpstag table (dt datetime, voltage int, gsm_sen int, mv int, ma int, mah int )
	declare @tag_exits char = case when EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'rf_message_tag_00'+ @id+'_' + @year) then '1' else '0' end

	declare @sql as nvarchar(max) 
    
#     {'begin_gpsdate': '20230301', 'end_gpsdate': '20230316', 'begin_gpstime': '000000', 'end_gpstime': '235959'}
	
    declare @begin_gpsdate nvarchar(10) = convert(varchar, convert(date,concat(@year,'/',@begin_m,'/',@begin_d)), 112)
	declare @end_gpsdate nvarchar(10) = convert(varchar, convert(date,concat(@year,'/',@end_m,'/',@end_d)), 112)
	declare @begin_gpstime nvarchar(10) = replace(convert(varchar, convert(time, concat(@begin_h,':0:0')), 108),':','')
	declare @end_gpstime nvarchar(10) = replace(convert(varchar, convert(time, concat(@end_h,':59:59')), 108),':','')

    
	set @sql = case when @tag_exits ='1' then 
		'select * from (
			select convert(datetime, concat(
			datepart(YEAR, dt), ''/'' ,
			datepart(MONTH, dt), ''/'' ,
			datepart(DAY, dt)  , '' '',
			datepart(HOUR, dt) , '':'',
			(datepart(MINUTE, dt) / 10), ''0'')) as dt , avg(voltage) as voltage, avg(gsm_sen) as gsm_sen, avg(mv) as mv, avg(ma) as ma, avg(mah) as mah
			from (
				select convert(datetime,
					concat(
						gpsdate,'' '', 
						substring(gpstime, 1,2),'':'',
						substring(gpstime, 3,2),'':'',
						substring(gpstime, 5,2))
				) as dt, voltage, gsm_sen, mv, ma, mah 
				from rf_message_gps_00' + @id +'_' + @year +' 
				where (
					gpsdate > ' + @begin_gpsdate + ' 
					or 
					( gpsdate = ' + @begin_gpsdate + ' and gpstime >= ' + @begin_gpstime + ')
				) and ( 
					gpsdate < ' + @end_gpsdate + '
					or 
					( gpsdate = ' + @end_gpsdate + ' and gpstime <= ' + @end_gpstime + ')
				)
				union
				select 
					convert(datetime,
						concat(
							' + @year + ', 
							RIGHT(''00''+ CAST(t_month  AS VARCHAR(2)),2), 
							RIGHT(''00''+ CAST(t_day    AS VARCHAR(2)),2),'' '', 
							RIGHT(''00''+ CAST(t_hour   AS VARCHAR(2)),2),'':'', 
							RIGHT(''00''+ CAST(t_minute AS VARCHAR(2)),2),'':'', 
							RIGHT(''00''+ CAST(t_sec    AS VARCHAR(2)),2))
					) as dt , voltage, null as gsm_sen, null as mv, null as ma, null as mah
				from rf_message_tag_00' + @id +'_' + @year +' 
				where
				(
					t_month > ' + @begin_m + '
					or 
					(t_month = ' + @begin_m + ' and t_day >' + @begin_d + ') 
					or 
					(t_month = ' + @begin_m + ' and t_day =' + @begin_d + ' and t_hour >=' + @begin_h + ') 
				)
				and (
					t_month < ' + @end_m + '
					or 
					(t_month = ' + @end_m + ' and t_day <' + @end_d + ') 
					or 
					(t_month = ' + @end_m + ' and t_day =' + @end_d + ' and t_hour <=' + @end_h + ') 
				)
			) as a 
			group by 
			datepart(YEAR, dt),
			datepart(MONTH, dt),
			datepart(DAY, dt),
			datepart(HOUR, dt),
			(datepart(MINUTE, dt) / 10) 
		) as b 
		order by dt'
	else
		'select * from (
			select convert(datetime, concat(
			datepart(YEAR, dt), ''/'' ,
			datepart(MONTH, dt), ''/'' ,
			datepart(DAY, dt)  , '' '',
			datepart(HOUR, dt) , '':'',
			(datepart(MINUTE, dt) / 10), ''0'')) as dt , avg(voltage) as voltage, avg(gsm_sen) as gsm_sen, avg(mv) as mv, avg(ma) as ma, avg(mah) as mah
			from (
				select convert(datetime,
					concat(
						gpsdate,'' '', 
						substring(gpstime, 1,2),'':'',
						substring(gpstime, 3,2),'':'',
						substring(gpstime, 5,2))
				) as dt, voltage, gsm_sen, mv, ma, mah 
				from rf_message_gps_00' + @id +'_' + @year +' 
				where (
					gpsdate > ' + @begin_gpsdate + ' 
					or 
					( gpsdate = ' + @begin_gpsdate + ' and gpstime >= ' + @begin_gpstime + ')
				) and ( 
					gpsdate < ' + @end_gpsdate + '
					or 
					( gpsdate = ' + @end_gpsdate + ' and gpstime <= ' + @end_gpstime + ')
				)
			) as a 
			group by 
			datepart(YEAR, dt),
			datepart(MONTH, dt),
			datepart(DAY, dt),
			datepart(HOUR, dt),
			(datepart(MINUTE, dt) / 10) 
		) as b 
		order by dt'
		end
	exec(@sql)
END

### 範例2 - 充電

In [ ]:
declare @id nvarchar(5) = {{ id }}
declare @begin date = {{ begin }}
declare @end date = {{ end }}


declare @temp table(dt datetime, voltage float, gsm_sen float, mv int, ma int, mah int)
insert into @temp exec dbo.tag_battery @begin, @end, @id

select 
    row_number() over(order by dt) as serial_id, 
    dt, 
    voltage 
into #TempTable2 
from @temp

declare @serial_id int;
declare @bottom_dt datetime  = (select top 1 dt from #TempTable2 order by serial_id )
declare @bottom_voltage int  = (select top 1 voltage from #TempTable2 order by serial_id )
declare @pre_voltage int = @bottom_voltage
declare @pre_dt datetime = @bottom_dt
declare @charge tinyint = 0
DECLARE @out TABLE (
start_dt datetime, 
start_v int,
end_dt datetime, 
end_v int, 
charge_v int, 
charge_h float
)
while ( select count(*) from #TempTable2) >0 
	begin 
	     declare @voltage int 
	     declare @dt datetime 
		 Select Top 1 @serial_id = serial_id, @dt = dt, @voltage =voltage  From #TempTable2 order by serial_id 

		 if @voltage > @pre_voltage and @charge = 0 
			begin 
			  set @bottom_voltage = @pre_voltage
			  set @bottom_dt = @pre_dt
			  set @charge = 1
			end 
		 if ((@voltage < @pre_voltage or @pre_voltage =100 ) and @charge = 1)
            begin
				insert into @out (start_dt,start_v,end_dt,end_v, charge_v,charge_h ) select @bottom_dt,@bottom_voltage, @pre_dt, @pre_voltage, @pre_voltage-@bottom_voltage , round(cast(DATEDIFF(MINUTE, @bottom_dt, @pre_dt)as float) / cast(60 as float),2)
				set @charge = 0
			end
		 if ( ( select count(*) from #TempTable2) = 1 and @charge = 1)
			begin 
				insert into @out (start_dt,start_v,end_dt,end_v, charge_v,charge_h ) select @bottom_dt,@bottom_voltage, @pre_dt, @pre_voltage, @pre_voltage-@bottom_voltage, round(cast(DATEDIFF(MINUTE, @bottom_dt, @pre_dt)as float) / cast(60 as float),2)			
			end
		 set  @pre_voltage = @voltage
		 set @pre_dt = @dt
		 Delete #TempTable2 Where serial_id = @serial_id
    end
drop table #TempTable2

select start_dt as '起始時間', end_dt as '結束時間', concat(start_v,' -> ',end_v) as '電壓' , charge_v as '充電電壓', charge_h as '耗時(h)', round(cast(charge_v as float) / cast(charge_h as float), 2)  as '充電率(電壓/小時)' from @out

### 範例三 - 放電

In [ ]:
declare @id NVARCHAR(5) = {{ id }}
declare @begin date = {{ begin }}
declare @end date = {{ end }}
declare @max_mah int = {{ max_mah }}

declare @temp table(dt datetime, voltage float, gsm_sen float, mv int, ma int, mah int)
insert into @temp exec dbo.tag_battery @begin, @end, @id

select 
    row_number() over(order by dt) as serial_id, 
    dt, 
    voltage 
into #TempTable2 
from @temp

declare @serial_id int;
declare @top_dt datetime  = (select top 1 dt from #TempTable2 order by serial_id )
declare @top_voltage int  = (select top 1 voltage from #TempTable2 order by serial_id )
declare @pre_voltage int = @top_voltage
declare @pre_dt datetime = @top_dt
declare @charge tinyint = 1
DECLARE @out TABLE (
start_dt datetime, 
start_v int,
end_dt datetime, 
end_v int,
drain_v int, 
duration_h float,
health float
)
while ( select count(*) from #TempTable2) >0 
	begin 
	     declare @voltage int 
	     declare @dt datetime 
		 Select Top 1 @serial_id = serial_id, @dt = dt, @voltage =voltage  From #TempTable2 order by serial_id 

		 if @voltage > @pre_voltage and @charge = 0 
			begin
				declare @health float 
				select top 1 @health = (mah *100*100) / (voltage*@max_mah) from @temp where ma is not null and dt >= @top_dt and dt <= @pre_dt
				insert into @out (start_dt,start_v,end_dt,end_v,drain_v,duration_h, health) select @top_dt,@top_voltage, @pre_dt, @pre_voltage, @top_voltage-@pre_voltage , round(cast(DATEDIFF(MINUTE, @top_dt, @pre_dt)as float) / cast(60 as float),2), @health
				set @charge = 1 
			end
		 if @voltage < @pre_voltage and @charge =1 
		    begin 
			  set @top_voltage = @pre_voltage
			  set @top_dt = @pre_dt
			  set @charge = 0
			end 
		 if ( ( select count(*) from #TempTable2) = 1 and @charge = 0)
			begin 
				select top 1  @health = (mah *100*100) / (voltage*@max_mah)  from @temp where ma is not null and dt >= @top_dt and dt <= @pre_dt
				insert into @out (start_dt,start_v,end_dt,end_v,drain_v,duration_h, health ) select @top_dt,@top_voltage, @pre_dt, @pre_voltage, @top_voltage-@pre_voltage , round(cast(DATEDIFF(MINUTE, @top_dt, @pre_dt)as float) / cast(60 as float),2), @health		
			end
		 set  @pre_voltage = @voltage
		 set @pre_dt = @dt
		 Delete #TempTable2 Where serial_id = @serial_id
    end
drop table #TempTable2
select start_dt as '起始時間', end_dt as '結束時間',  concat(start_v,' -> ',end_v) as '電壓', drain_v as '消耗電壓', duration_h as '耗時(h)', round(cast(drain_v as float) / cast(duration_h as float), 2)  as '耗電率(電壓/小時)', health as '健康度' from @out

In [4]:
2838*100*100/(95.9*3200)

92.47914494264859

In [6]:
2874*100*100/(100*3200)

89.8125